# Raw data analysis
Plots summary of channel ADC values and trigger rates

## Paths
Set ``geometrypath`` to point to the larpix-geometry yaml file you'd like to use to plot x,y positions of pixels
Set ``datapath`` to point to the directory containing datafiles you'd like to look at

In [72]:
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import os
import glob
import h5py
from collections import defaultdict
import time
import yaml
from matplotlib import cm

In [73]:
geometrypath = '/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-software/larpix-geometry/larpixgeometry/layouts/layout-2.4.0.yaml'

# datapath = '/global/project/projectdirs/dune/data/larpix/raw_data/raw_20_10_12/500_V_cm'
# datapath = '/global/project/projectdirs/dune/data/module_0/tile_10x10/'

datapath = '/global/project/projectdirs/dune/www/data/Module0-HV/LArPix/dataRuns/rawData/'
# datapath = '/global/project/projectdirs/dune/www/data/Module0-HV/LArPix/pedestalRuns/'
# datapath = '/global/project/projectdirs/dune/www/data/Module0-HV/LArPix/leakageCurrent/'

# datapath = '/global/project/projectdirs/dune/www/data/Bern-singlecube/LArPix/dataRuns/rawData' # data runs
# datapath = '/global/project/projectdirs/dune/www/data/Bern-singlecube/LArPix/pedestalRuns' # pedestal runs
# datapath = '/global/project/projectdirs/dune/www/data/Bern-singlecube/LArPix/leakageCurrent' # pedestal runs

In [74]:
with open(geometrypath) as fi:
    geo = yaml.full_load(fi)
chip_pix = dict([(chip_id, pix) for chip_id,pix in geo['chips']])

## XY hit map

In [75]:
data_cache = dict()

In [76]:
%matplotlib widget
files = sorted([os.path.basename(path) for path in glob.glob(datapath+'/*_.h5')])

def unique_channel_id(d):
    return ((d['io_group'].astype(int)*256 + d['io_channel'].astype(int))*256 \
            + d['chip_id'].astype(int))*64 + d['channel_id'].astype(int)
def unique_channel_id_2_str(unique_id,*args,**kwargs):
    return (unique_id//(256*256*64)).astype(int).astype(str) \
        + '-' + ((unique_id//(256*64))%256).astype(int).astype(str) \
        + '-' + ((unique_id//64)%256).astype(int).astype(str) \
        + '-' + (unique_id%64).astype(int).astype(str)
def unique_chip_id(d):
    return (d['io_group'].astype(int)*256 + d['io_channel'].astype(int))*256 \
            + d['chip_id'].astype(int)

@widgets.interact
def display(filenames=widgets.SelectMultiple(options=list(reversed(sorted(files))), rows=10, 
                                             description="File")
           ):   
    plt.close('all')
    fig1 = None
    for filename in filenames:
        if not filename in data_cache:
            print('opening',filename,'...')
            f = h5py.File(os.path.join(datapath,filename),'r')
            unixtime = f['packets']['timestamp'][f['packets']['packet_type'] == 4]
            livetime = np.max(unixtime) - np.min(unixtime)

            data_mask = f['packets']['packet_type'] == 0
            data_mask = np.logical_and(f['packets']['valid_parity'], data_mask)
            dataword = f['packets']['dataword'][data_mask]
            
            if 'configs' in f.keys():
                configs_unique_chip_id = unique_chip_id(f['configs'])
            else:
                configs_unique_chip_id = np.zeros((1,))-1
            
            unique_id = unique_channel_id(f['packets'][data_mask])
            unique_id_set = np.unique(unique_id)
            d = defaultdict(dict)
            last = time.time()
            for i,id in enumerate(unique_id_set):
                if time.time() > last + 1:
                    print('{}/{} {}'.format(i+1,len(unique_id_set),unique_channel_id_2_str(id)),end='\r')
                    last = time.time()
                id_mask = unique_id == id
                config_mask = configs_unique_chip_id == id//64
                if np.sum(id_mask) < 3:
                    continue
                masked_dataword = dataword[id_mask]
                if np.any(config_mask):
                    d[id]['trim'] = f['configs']['registers'][config_mask,id%64][-1]
                    d[id]['threshold'] = f['configs']['registers'][config_mask,64][-1]
                    d[id]['threshold_88K_mV'] = d[id]['threshold']*1800/256 + 465 + 2.34 * d[id]['trim']
                    d[id]['threshold_300K_mV'] = d[id]['threshold']*1800/256 + 210 + 1.45 * d[id]['trim']
                else:
                    d[id]['trim'] = 0
                    d[id]['threshold'] = 0
                    d[id]['threshold_88K_mV'] = 0
                    d[id]['threshold_300K_mV'] = 0
                d[id]['min'] = np.min(masked_dataword)
                d[id]['mean'] = np.mean(masked_dataword)
                d[id]['med'] = np.median(masked_dataword)
                d[id]['std'] = np.std(masked_dataword)
                d[id]['rate'] = len(masked_dataword) / (livetime + 1e-9)
                pix = chip_pix[(id//64)%256][id%64] if (id//64)%256 in chip_pix else None
                if pix:
                    d[id]['x'] = geo['pixels'][pix][1]
                    d[id]['y'] = geo['pixels'][pix][2]
                else:
                    d[id]['x'] = 0.
                    d[id]['y'] = 0.
            data_cache[filename] = d
        else:
            print('loading',filename,'from cache')
            d = data_cache[filename]

        if not fig1:
            fig1,axes = plt.subplots(3,1,sharex='col',num='summary 1',figsize=(8,6))
        else:
            fig1 = plt.figure('summary 1')
            axes = fig1.axes
        axes[0].scatter([key for key in d if 'mean' in d[key]],
                        [d[key]['mean'] for key in d if 'mean' in d[key]],
                        marker='.',alpha=0.5)
        axes[1].scatter([key for key in d if 'std' in d[key]],
                        [d[key]['std'] for key in d if 'std' in d[key]],
                        marker='.',alpha=0.5)
        axes[2].scatter([key for key in d if 'rate' in d[key]],
                        [d[key]['rate'] for key in d if 'rate' in d[key]],
                        marker='.',alpha=0.5)
        axes[2].set(xlabel='unique channel')
        axes[0].set(ylabel='mean ADC')
        axes[1].set(ylabel='std ADC')
        axes[2].set(ylabel='rate [Hz]')
        for ax in axes:
            ax.grid(1)
        axes[2].set_yscale('log')

        ax2 = axes[0].secondary_xaxis('top', functions=(lambda x: x, lambda x: x))
        ax2.xaxis.set_major_formatter(ticker.FuncFormatter(unique_channel_id_2_str))
        ax2.set(xlabel='channel key')
        plt.legend(range(len(filenames)))
        plt.tight_layout()

        fig2, axes = plt.subplots(3,1,sharex='col',sharey='col',num='summary 2 {}'.format(filename),
                                  figsize=(6,12))
        x = np.array([d[key]['x'] for key in d if 'x' in d[key]])
        y = np.array([d[key]['y'] for key in d if 'y' in d[key]])
        c0 = fig2.colorbar(axes[0].scatter(x,y,c=[d[key]['mean'] for key in d if 'mean' in d[key]], 
                                           marker='.', alpha=0.5*2), ax=axes[0])
        c1 = fig2.colorbar(axes[1].scatter(x,y,c=[d[key]['std'] for key in d if 'std' in d[key]], 
                                           marker='.', norm=colors.LogNorm(), alpha=0.5*2), ax=axes[1])
        c2 = fig2.colorbar(axes[2].scatter(x,y,c=[d[key]['rate'] for key in d if 'rate' in d[key]], 
                                           marker='.', norm=colors.LogNorm(), alpha=0.5*2), ax=axes[2])
        axes[2].set(xlabel='x [mm]')
        axes[0].set(ylabel='y [mm]',title=filename)
        c0.set_label('mean ADC')
        axes[1].set(ylabel='y [mm]')
        c1.set_label('std ADC')
        axes[2].set(ylabel='y [mm]')
        c2.set_label('rate [Hz]')

        ax2 = axes[0].secondary_xaxis('top', functions=(lambda x: x, lambda x: x))
        ax2.set(xlabel='x [mm]')
        plt.tight_layout()
        
        rates = np.array([d[key]['rate'] for key in d if 'rate' in d[key]])
        keys = np.array([key for key in d if 'rate' in d[key]])
        idx_sorted = np.argsort(rates)
        for i in idx_sorted[-10:]:
            print(unique_channel_id_2_str(keys[i]),'trigger rate',rates[i])
        
        if any(['threshold_88K_mV' in d[key] for key in d]):
            fig3, axes = plt.subplots(3,1,sharex='col',sharey='col',num='summary 3 {}'.format(filename),
                                      figsize=(6,12))
            x = np.array([d[key]['x'] for key in d if 'x' in d[key]])
            y = np.array([d[key]['y'] for key in d if 'y' in d[key]])
            c0 = fig3.colorbar(axes[0].scatter(x,y,c=[d[key]['trim'] for key in d if 'trim' in d[key]], 
                                               marker='.', alpha=0.5*2), ax=axes[0])
            c1 = fig3.colorbar(axes[1].scatter(x,y,c=[d[key]['threshold'] for key in d 
                                                      if 'threshold' in d[key]], 
                                               marker='.', #norm=colors.LogNorm(), 
                                               alpha=0.5*2), ax=axes[1])
            c2 = fig3.colorbar(axes[2].scatter(x,y,c=[d[key]['threshold_88K_mV'] for key in d 
                                                      if 'threshold_88K_mV' in d[key]], 
                                               marker='.', #norm=colors.LogNorm(), 
                                               alpha=0.5*2), ax=axes[2])
            axes[2].set(xlabel='x [mm]')
            axes[0].set(ylabel='y [mm]',title=filename)
            c0.set_label('pixel trim')
            axes[1].set(ylabel='y [mm]')
            c1.set_label('global threshold')
            axes[2].set(ylabel='y [mm]')
            c2.set_label('threshold [mV]')

            ax2 = axes[0].secondary_xaxis('top', functions=(lambda x: x, lambda x: x))
            ax2.set(xlabel='x [mm]')
            plt.tight_layout()


interactive(children=(SelectMultiple(description='File', options=('datalog_2020_12_01_08_39_46_CET_.h5', 'data…

## Plot trigger rates

In [77]:
rate_cache = {}

In [78]:
@widgets.interact
def display(filenames=widgets.SelectMultiple(options=list(reversed(sorted(files))), rows=10, 
                                             description="File")
           ):   
    plt.close('all')
    fig1 = None
    for filename in filenames:
        if not filename in rate_cache:
            rate_cache[filename] = dict()
            print('opening',filename,'...')
            f = h5py.File(os.path.join(datapath,filename),'r')
            if np.any(f['packets']['packet_type'] == 4):
                timestamp_packet_mask = f['packets']['packet_type'] == 4
                timestamp_idcs = np.argwhere(timestamp_packet_mask).flatten()
                message_groups = np.split(f['packets']['timestamp'], timestamp_idcs)
                unixtime = np.array([grp[0] for grp in message_groups for _ in range(0,len(grp))])
                unixtimes = np.arange(np.min(unixtime),np.max(unixtime))

                packet_type_mask = [f['packets']['packet_type'] == packet_type for packet_type in range(8)]
                for packet_type in range(8):
                    rate_cache[filename]['packet count {}'.format(packet_type)] = np.histogram(unixtime[packet_type_mask[packet_type]], 
                                                                                               bins=unixtimes)[0]
                rate_cache[filename]['packet count all'] = np.histogram(unixtime, bins=unixtimes)[0]
                
                parity_error_mask = f['packets'][packet_type_mask[0]]['valid_parity'] == 0
                rate_cache[filename]['parity errors'] = np.histogram(unixtime[packet_type_mask[0]][parity_error_mask], bins=unixtimes)[0]
                
                shared_fifo_mask = [f['packets'][packet_type_mask[0]]['shared_fifo'] == full for full in range(0,4)]
                local_fifo_mask = [f['packets'][packet_type_mask[0]]['local_fifo'] == full for full in range(0,4)]
                for full in range(0,4):
                    rate_cache[filename]['shared fifo {}'.format(full)] = np.histogram(unixtime[packet_type_mask[0]][shared_fifo_mask[full]], 
                                                                                       bins=unixtimes)[0]
                    rate_cache[filename]['local fifo {}'.format(full)] = np.histogram(unixtime[packet_type_mask[0]][local_fifo_mask[full]], 
                                                                                       bins=unixtimes)[0]
                
                rate_cache[filename]['time'] = unixtimes
                
                data_mask = rate_cache[filename]['packet count 0'] > 0
                rate_cache[filename]['hit idx'] = [
                    np.argwhere(unixtime == unixtimes[:-1][data_mask][i]).flatten()
                    for i in range(np.sum(data_mask))
                ]
            else:
                del rate_cache[filename]
    
    for filename in reversed(sorted(rate_cache)):
        if filename in filenames:
            plt.close('rates {}'.format(filename))
            fig_agg2, axes = plt.subplots(4,1,sharex='col',num=filename,figsize=(6,12))

            x = rate_cache[filename]['time'][:-1]
            for packet_type in range(8):
                y = rate_cache[filename]['packet count {}'.format(packet_type)]
                axes[0].plot(x,y,'.-',alpha=0.5,label='packet type = {}'.format(packet_type))
            y = rate_cache[filename]['packet count all']
            axes[0].plot(x,y,'k-',alpha=0.5,label='all')
#             axes[0].set_xlabel('timestamp')
            axes[0].set_ylabel('rate [Hz]')
            axes[0].set_yscale('log')
            axes[0].grid(1)
            axes[0].legend(fontsize='xx-small')
            
            for packet_type in range(8):
                y = rate_cache[filename]['packet count {}'.format(packet_type)] / rate_cache[filename]['packet count all']
                axes[1].plot(x,y,'.-',alpha=0.5,label='packet type = {}'.format(packet_type))
#             axes[1].set_xlabel('timestamp')
            axes[1].set_ylabel('fraction')
            axes[1].grid(1)
            axes[1].set_yscale('log')
            axes[1].legend(fontsize='xx-small')
            
            for full in range(4):
                y = rate_cache[filename]['shared fifo {}'.format(full)] / rate_cache[filename]['packet count 0']
                axes[2].plot(x,y,'.-',alpha=0.5,label='shared fifo = {}'.format(full))
                y = rate_cache[filename]['local fifo {}'.format(full)] / rate_cache[filename]['packet count 0']
                axes[2].plot(x,y,'.-',alpha=0.5,label='local fifo = {}'.format(full))
#             axes[2].set_xlabel('timestamp')
            axes[2].set_ylabel('fraction')
            axes[2].grid(1)
            axes[2].set_yscale('log')
            axes[2].legend(fontsize='xx-small')

            y = rate_cache[filename]['parity errors'] / rate_cache[filename]['packet count 0']
            axes[3].plot(x,y,'.-',alpha=0.5,label='parity errors')
            axes[3].set_xlabel('timestamp')
            axes[3].set_ylabel('fraction')
            axes[3].grid(1)
            axes[3].set_yscale('log')
            axes[3].legend(fontsize='xx-small')
            
            plt.tight_layout()
            
            options = [(time,i) for i,time in enumerate(rate_cache[filename]['time'][:-1][rate_cache[filename]['packet count 0'] > 0])]
            display = None
            @widgets.interact
            def display(
                filename=filename,
                timestamp_idxs=widgets.SelectMultiple(
                        options=options,
                        value=[],
                        rows=5,
                        description='Hit map for',
                        disabled=False
                    )
                ):
                try:
                    if len(timestamp_idxs):
                        f = h5py.File(os.path.join(datapath,filename),'r')
                        idxs = np.concatenate([rate_cache[filename]['hit idx'][timestamp_idx] for timestamp_idx in timestamp_idxs])
                        chip_ids = f['packets']['chip_id'][idxs]
                        channel_ids = f['packets']['channel_id'][idxs]
                        packet_types = f['packets']['packet_type'][idxs]
                        pix = np.array([
                            chip_pix[chip_id][channel_id] if chip_id in chip_pix else -1
                            for chip_id, channel_id in zip(chip_ids[packet_types==0],channel_ids[packet_types==0])
                        ])
                        x = np.array([geo['pixels'][p][1] for p in pix if p != -1 and p is not None])
                        y = np.array([geo['pixels'][p][2] for p in pix if p != -1 and p is not None])

                        plt.close('{} xy rate map'.format(filename))
                        fig = plt.figure('{} xy rate map'.format(filename))
                        plt.hist2d(x, y, weights=[1./len(timestamp_idxs)]*len(y),
                                   bins=(np.linspace(-157.407,157.407,71),np.linspace(-157.407,157.407,71)),
                                   norm=colors.LogNorm())
                        cb = plt.colorbar()
                        plt.xlabel('x [mm]')
                        plt.ylabel('y [mm]')
                        cb.set_label('trigger rate [Hz]')
                        plt.tight_layout()
                except Exception as e:
                    print(e)
            

interactive(children=(SelectMultiple(description='File', options=('datalog_2020_12_01_08_39_46_CET_.h5', 'data…

## Plot test pulse summary

In [7]:
import numpy as np
import matplotlib.pyplot as plt
dac        = [13,3,0,15,14,12]
responding = [0.96,0.84,0.21,0.95,0.95,0.95]
high_eff   = [0.82,0.58,0.02,0.83,0.81,0.81]
eff        = [0.90,0.74,0.06,0.91,0.90,0.90]
noise      = [0.03,0.03,0.03,0.03,0.03,0.03]
data = np.array(sorted(zip(dac,responding,high_eff,eff,noise)))

plt.close('test pulse data')
plt.figure('test pulse data')
plt.plot(data[:,0],data[:,1],'r.-',label='responding')
plt.plot(data[:,0],data[:,2],'b.-',label='>85% efficiency')
plt.plot(data[:,0],data[:,3],'k.-',label='overall')
plt.plot(data[:,0],data[:,4],'g.-',label='cross talk')
plt.legend()
plt.xlabel('DAC [1.8ke-]')
plt.ylabel('trigger efficiency')
plt.ylim(0,1)
plt.xlim(0,16)
plt.grid()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …